# 💡 MosaicPi_Guide: **"Linear Algebra in MosaicPi"**

> All code and examples are shared to help researchers, students, and engineers understand the reasoning behind DDDA — and to make it easy to apply dimensional analysis to your own data.  
> This notebook serves as an entry-level guide for teaching, validating physical models, and enabling domain-specific knowledge engineering through data-driven dimensional reasoning.

---

## 🎯 What You'll Learn

**MosaicPi中的线性代数应用**

This notebook introduces the **fundamental concepts of linear algebra** that are widely used in applied mathematics, data science, physics, and engineering. The focus is on building an intuitive understanding alongside practical computations using NumPy.

1. **物理模型，隐函数，流形**  
   Understand why we reduce variables and how dimensional consistency enables model generalization.

2. **变量组合**  
   Encode physical units of input quantities using base units and build the D-matrix.

3. **变量组合评估**  
   Discover dimensionless groups by solving linear algebraic equations on the D-matrix.

4. **显式化策略可视化**  
   Learn to assess whether derived groups make physical and computational sense.

5. **不确定性定量化**  
   Set the stage for further steps in the DDDA pipeline including Pi-group selection, uncertainty quantification, and regime detection.

---

## 👤 Author

- **Name**: Jiashun Pang  
- **Created**: August 2025  
- **Affiliation**: MosaicPi, open research notebook  
- **Notebook Focus**:  
  A hands-on exploration of dimensional analysis — from aggregated raw quantities to symbolic Pi-group discovery and preparation for downstream DDDA tasks.

---

📌 *This notebook is designed to be accessible for learners new to dimensional analysis, while also laying the foundation for advanced applications in the full MosaicPi pipeline.*

---

# 5. 主成分分析（PCA）

## 5.1 背景与目标

* 目的：在高维数据中找出方差最大的正交方向，用更少的线性组合表示主要信息。
* 应用关键词：降维、去噪、可视化、变量筛选、前处理（为后续显函数/建模做准备）。

## 5.2 数据准备与协方差矩阵

* 数据矩阵约定：$X\in\mathbb{R}^{n\times d}$（$n$ 个样本，$d$ 个变量/特征）。
* 居中与标准化：

  * 居中：$X_c = X - \mathbf{1}\mu^\top$，$\mu=\frac{1}{n}\sum_i x_i$。
  * 视单位量纲是否一致决定是否标准化（z-score）。
* 协方差矩阵：

  $$
  C=\frac{1}{n-1}X_c^\top X_c\ \in\mathbb{R}^{d\times d}
  $$

  * 可加权版本（非均匀权重/重复测量）：$C=\frac{1}{\sum w_i}X_c^\top W X_c$。

## 5.3 特征值分解 vs SVD

* 特征值分解（EVD）：

  $$
  C = V\,\Lambda\,V^\top,\quad \Lambda=\mathrm{diag}(\lambda_1\ge\cdots\ge\lambda_d\ge0)
  $$

  * 主轴（loadings）：列向量 $v_j$；
  * 方差解释量：$\lambda_j$。
* SVD（数值更稳健，适合 $n\ll d$ 或大规模）：

  $$
  X_c = U\,\Sigma\,V^\top,\quad C=\frac{1}{n-1}V\,\Sigma^2\,V^\top
  $$

  * $V$ 与 EVD 的特征向量一致；$\lambda_j=\sigma_j^2/(n-1)$。

## 5.4 降维思想与主成分

* 取前 $k$ 个最大特征值的方向：$V_k=[v_1,\dots,v_k]$。
* 得到主成分（得分）：

  $$
  Z = X_c\,V_k\ \in\mathbb{R}^{n\times k}
  $$
* 解释方差比例（EVR）与累计解释方差（CER）：

  $$
  \text{EVR}_j=\frac{\lambda_j}{\sum_i\lambda_i},\quad \text{CER}_k=\sum_{j=1}^k\text{EVR}_j
  $$
* 白化（可选）：$Z_{\text{white}}=X_c\,V_k\,\Lambda_k^{-1/2}$。

## 5.5 在实验数据中的应用

### A. 高维隐函数的显式化 → 降维预处理

* 估计内在维度：看碎石图/拐点或 CER 阈值（如 90–99%）。
* 在 $Z$ 空间做局部/全局建模（回归、IFT 判定），减少冗余共线性。
* 局部 PCA（滑动窗口/聚类后分块）帮助处理流形弯曲与非线性区域化。

### B. 噪声过滤（低秩近似）

* 低秩重构：

  $$
  \hat{X}=U_k\,\Sigma_k\,V_k^\top + \mathbf{1}\mu^\top
  $$
* 选 $k$ 的准则：EVR 阈值、交叉验证的重构误差、噪声门限（奇异值硬/软阈值）、稳健 PCA（分解为低秩+稀疏）。

### C. 变量选择指导（传感器/特征挑选）

* 载荷矩阵 $V_k$：$|v_{j,i}|$ 大的变量对第 $j$ 主成分贡献大。
* 可视化：biplot / loading heatmap；基于贡献度排序与分组（去冗余）。
* 拓展：Sparse PCA（稀疏可解释），Varimax（可解释旋转），Active Subspaces（基于梯度的主方向）。

## 5.6 实践流程（可落地的步骤）

1. 单位与量纲检查；决定是否做 z-score 标准化。
2. 计算 $X_c$，做 SVD（推荐）或对 $C$ 做 EVD。
3. 画碎石图与 CER 曲线，选 $k$。
4. 得分 $Z=X_c V_k$，用于后续建模/显函数化/聚类。
5. 若去噪：重构 $\hat{X}=U_k\Sigma_k V_k^\top+\mu$。
6. 若变量选择：分析 $V_k$ 的绝对载荷，出排名与分组建议。
7. 评估：重构误差、下游模型性能（如 IFT 判定的稳健性/稳定域面积）。

## 5.7 注意事项与陷阱

* **尺度敏感**：未标准化时，大量纲变量主导方向。
* **非线性结构**：PCA 只捕捉线性子空间；可考虑 Kernel PCA/Isomap/UMAP。
* **异常值**：对奇异值敏感；用稳健 PCA（M-estimator/Huber/median-of-means）。
* **缺失值**：EM-PCA/低秩补全。
* **符号不唯一**：$v$ 与 $-v$ 等价；报告时统一符号规范。
* **时间序列/相关性**：若强自相关，先差分/滤波或用时延嵌入的协方差。

## 5.8 扩展与对比

* Kernel PCA（非线性映射）、Incremental/Online PCA（流数据）、Randomized SVD（大规模）、Robust PCA（低秩+稀疏）。
* 与 PLS 的区别：PCA 无监督只看 $X$ 方差；PLS 监督式寻找与响应 $y$ 相关的方向。

## 5.9 常用指标与可视化

* 碎石图、累计解释方差、biplot、载荷热图、重构误差曲线、残差分析（Q-Q 图）。

---
